In [1]:
import csv
import os
import pandas as pd

# Define the CSV file paths
results_data_path = os.path.expanduser('~/PiEEG_Miniaturization-Model_Compression_Fat/model_optimization_tests/results')
csv_file = os.path.join(results_data_path, 'results_data.csv') 
output_file = os.path.join(results_data_path, 'compression_results.csv')

# Columns to select
columns_to_select = ['subject', 'model_name', 'sparsity', 'quantization', 'model_size', 'test_acc', 'compression_ratio']

# Subjects of interest
subjects_of_interest = list(range(1, 34))

# Model names of interest
model_names_of_interest = ['deep_conv_net', 'eeg_net', 'shallow_conv_net']

# Sparsities of interest
sparsities_of_interest = [0.0, 0.1, 0.3, 0.5, 0.7, 0.9]

# Read the existing New_results.csv file into a DataFrame if it exists
if os.path.exists(output_file):
    existing_df = pd.read_csv(output_file)
else:
    existing_df = pd.DataFrame(columns=columns_to_select)

# Read the CSV file into a DataFrame
df = pd.read_csv(csv_file)

# Initialize an empty list to store new results
new_results = []

# Iterate through each subject, model, and sparsity
for subject in subjects_of_interest:
    for model_name in model_names_of_interest:
        # Filter for the current subject, model, and sparsity 0.0
        original_model = df[
            (df['subject'] == subject) &
            (df['model_name'] == model_name) &
            (df['sparsity'] == 0.0)
        ]

        if len(original_model) == 0:
            continue
        
        # Get the original model size and test accuracy
        original_model_size = original_model.iloc[0]['model_size']
        original_test_acc = original_model.iloc[0]['test_acc']
        
        # Collect pruned model sizes for all specified sparsities
        for sparsity in sparsities_of_interest:
            pruned_model = df[
                (df['subject'] == subject) &
                (df['model_name'] == model_name) &
                (df['sparsity'] == sparsity)
            ]
            
            if len(pruned_model) == 0:
                continue
            
            # Get the pruned model size and test accuracy
            pruned_model_size = pruned_model.iloc[0]['model_size']
            test_acc = pruned_model.iloc[0]['test_acc']
            
            # Calculate compression ratio
            if sparsity == 0.0:
                compression_ratio = 1.0  # Compression ratio for sparsity 0.0 is 1.0
            else:
                compression_ratio = original_model_size / pruned_model_size
            
            # Append new result to the list
            new_results.append({
                'subject': subject,
                'model_name': model_name,
                'sparsity': sparsity,
                'quantization': '',  # Assuming quantization is empty or None
                'model_size': pruned_model_size,
                'test_acc': test_acc,
                'compression_ratio': compression_ratio
            })

# Convert new results to DataFrame
new_results_df = pd.DataFrame(new_results)

# Concatenate existing and new results DataFrames
final_df = pd.concat([existing_df, new_results_df], ignore_index=True)

# Save the concatenated DataFrame to the output file
final_df.to_csv(output_file, index=False)

print(f"New compression ratios appended and saved to {output_file}")

New compression ratios appended and saved to /home/paul/PiEEG_Miniaturization-Model_Compression_Fat/model_optimization_tests/results/compression_results.csv
